In [1]:
!python -V

Python 3.8.19


In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [3]:
import os

# Obter o diretório atual
current_dir = os.getcwd()

# Subir um nível no diretório
parent_dir = os.path.dirname(current_dir)

# Mudar para o diretório desejado
os.chdir(parent_dir)

In [4]:
df = pd.read_csv(parent_dir + '\\data\\bronze\\BRA_2012_2023.csv', sep = ',')

print(df.shape)
df.head()

(4560, 19)


,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
0,Brazil,Serie A,2012,19/05/2012,22:30,Palmeiras,Portuguesa,1.0,1.0,D,1.75,3.86,5.25,1.76,3.87,5.31,1.69,3.50,4.90
1,Brazil,Serie A,2012,19/05/2012,22:30,Sport Recife,Flamengo RJ,1.0,1.0,D,2.83,3.39,2.68,2.83,3.42,2.70,2.59,3.23,2.58
2,Brazil,Serie A,2012,20/05/2012,01:00,Figueirense,Nautico,2.0,1.0,H,1.60,4.04,6.72,1.67,4.05,7.22,1.59,3.67,5.64
3,Brazil,Serie A,2012,20/05/2012,20:00,Botafogo RJ,Sao Paulo,4.0,2.0,H,2.49,3.35,3.15,2.49,3.39,3.15,2.35,3.26,2.84
4,Brazil,Serie A,2012,20/05/2012,20:00,Corinthians,Fluminense,0.0,1.0,A,1.96,3.53,4.41,1.96,3.53,4.41,1.89,3.33,3.89


In [5]:
def public_definition(df):
    """
    Prepares the dataset by removing missing values, creating a target variable,
    and splitting the data into training and test sets.

    Parameters:
    df (DataFrame): The input DataFrame containing match data.

    Returns:
    DataFrame: The processed DataFrame with added 'Target' and 'is_test' columns.
    """
    # Remove linhas com valores ausentes e redefine o índice
    df_filtered = df.dropna().reset_index(drop=True)

    # Cria a variável alvo: 1 se o time da casa venceu, 0 caso contrário
    df_filtered['Target'] = np.where(df_filtered['Res'] == 'H', 1, 0)
    df_filtered['is_test'] = 0

    # Lista de temporadas únicas
    unique_seasons = df_filtered["Season"].unique()

    # Amostra 20% dos dados de cada temporada para o conjunto de teste
    for season in unique_seasons:
        season_indices = df_filtered[df_filtered['Season'] == season].index
        test_indices = np.random.choice(season_indices, size=int(round(len(season_indices) * 0.2)), replace=False)
        df_filtered.loc[test_indices, 'is_test'] = 1

    return df_filtered

In [6]:
abt = public_definition(df)

print(abt.shape)
abt.head()

(4559, 21)


,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,...,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,Target,is_test
0,Brazil,Serie A,2012,19/05/2012,22:30,Palmeiras,Portuguesa,1.0,1.0,D,...,3.86,5.25,1.76,3.87,5.31,1.69,3.50,4.90,0,0
1,Brazil,Serie A,2012,19/05/2012,22:30,Sport Recife,Flamengo RJ,1.0,1.0,D,...,3.39,2.68,2.83,3.42,2.70,2.59,3.23,2.58,0,1
2,Brazil,Serie A,2012,20/05/2012,01:00,Figueirense,Nautico,2.0,1.0,H,...,4.04,6.72,1.67,4.05,7.22,1.59,3.67,5.64,1,0
3,Brazil,Serie A,2012,20/05/2012,20:00,Botafogo RJ,Sao Paulo,4.0,2.0,H,...,3.35,3.15,2.49,3.39,3.15,2.35,3.26,2.84,1,0
4,Brazil,Serie A,2012,20/05/2012,20:00,Corinthians,Fluminense,0.0,1.0,A,...,3.53,4.41,1.96,3.53,4.41,1.89,3.33,3.89,0,0


In [7]:
abt.to_parquet(parent_dir + '\\data\\silver\\BRA_baseline.csv')